# Step 1: Identify seed 0 and seed 1 pages for economic recovery functional network 

This approach does not rely on the existing knowledge graph. A functional graph based on page hit session data is created, before further filtering of the graph, to find a list of pages related to the economic recovery whole user journey (WUJ). 

The first step is to identify seed 0 and seed 1 pages. 

In [50]:
import pandas as pd;
from apiclient import discovery
from httplib2 import Http
import oauth2client
from oauth2client import file, client, tools
from oauth2client.service_account import ServiceAccountCredentials
from pydrive.drive import GoogleDrive
from google.colab import auth

GOOGLE_APPLICATION_CREDENTIALS = os.getenv('GOOGLE_APPLICATION_CREDENTIALS')



gauth = GoogleAuth()
scope = ['https://www.googleapis.com/auth/drive']
gauth.credentials = ServiceAccountCredentials.from_json_keyfile_name(GOOGLE_APPLICATION_CREDENTIALS, scope)
gauth.credentials = GoogleCredentials.get_application_default()

drive = GoogleDrive(gauth)

my_file = drive.CreateFile({'id': '1x3lHpUkIm-KTnWrRJU11An1J2Yb4KJT8z0QpuXOm3e0'}) 
my_file.Upload()

ModuleNotFoundError: No module named 'google.colab'

In [46]:
from google.oauth2.service_account import Credentials
from googleapiclient.discovery import build

credentials = Credentials.from_service_account_file(json_key_filename, scopes=SCOPES)
service = build('sheets', 'v4', credentials=credentials)



ModuleNotFoundError: No module named 'auth'

In [ ]:
response